In [4]:
import xml.etree.ElementTree as ET
import csv

from classes.xml import values
from classes.vector3 import Vector3
from classes.quaternion import Quaternion
from classes.metamodel import Hull, Weapon, Link, MobileObject, WeaponWithMobileObjects, Cannon, WeaponWithCannons
from classes.mutation import mutate_model

# 1. Dataset construction

In [5]:
path_in = "models/"
objects = ["Argos", "Orion", "Teuthus", "Vermis"]

nodes = {"ObjectData": 0, "PhysicalObjectData": 1, "AliveObjectData": 2, "ComponentData": 3, "LinkData": 0}

path_out = "results/"
datasets = {}

## 1.1. Aggregation relationships (5)

### 1.1.1 Hull
### 1.1.2 Weapon
### 1.1.3 Link
### 1.1.4 AIUnit
### 1.1.5 MovementAI

In [6]:
# 1.1.1. Hull.

'''
1st step: Parsing of XML models.
'''

dataset = []

for o in objects:
    xml = ET.parse(f"{path_in}{o}.xml").getroot()

    print(f"###### Working with {o}.xml #######")

    ### Hulls ###
    hulls = xml.findall("HULL/Hull")
    print(f"\nNo. of 'HULL/Hull': %d\n" % len(hulls))

    if len(hulls) > 0:
        hull_types = list(map(int, [hull.get("HullType") for hull in hulls]))
        print(f"HullTypes: {hull_types}\n")
        
        # ObjectData
        scales_x = values(hulls, nodes["ObjectData"], "ScaleX")
        scales_y = values(hulls, nodes["ObjectData"], "ScaleY")
        scales_z = values(hulls, nodes["ObjectData"], "ScaleZ")
        scales = [Vector3(scales_x[i], scales_y[i], scales_z[i]) for i in range(len(hulls))]
        print(f"Scales: {scales}\n")

        positions_x = values(hulls, nodes["ObjectData"], "PositionX")
        positions_y = values(hulls, nodes["ObjectData"], "PositionY")
        positions_z = values(hulls, nodes["ObjectData"], "PositionZ")
        positions = [Vector3(positions_x[i], positions_y[i], positions_z[i]) for i in range(len(hulls))]
        print(f"Positions: {positions}\n")

        orientations_w = values(hulls, nodes["ObjectData"], "OrientationW")
        orientations_x = values(hulls, nodes["ObjectData"], "OrientationX")
        orientations_y = values(hulls, nodes["ObjectData"], "OrientationY")
        orientations_z = values(hulls, nodes["ObjectData"], "OrientationZ")
        orientations = [Quaternion(orientations_w[i], orientations_x[i], orientations_y[i], orientations_z[i]) for i in range(len(hulls))]
        print(f"Orientations: {orientations}\n")

        # PhysicalObjectData
        object_masses = list(map(float, values(hulls, nodes["PhysicalObjectData"], "ObjectMass")))
        print(f"ObjectMasses: {object_masses}\n")

        object_materials = list(map(int, values(hulls, nodes["PhysicalObjectData"], "ObjectMaterial")))
        print(f"ObjectMaterials: {object_materials}\n")

        # AliveObjectData
        object_maximum_lifes = list(map(float, values(hulls, nodes["AliveObjectData"], "ObjectMaximumLife")))
        print(f"ObjectMaximumLifes: {object_maximum_lifes}\n")

        object_armors = list(map(float, values(hulls, nodes["AliveObjectData"], "ObjectArmor")))
        print(f"ObjectArmors: {object_armors}\n")

        destruction_delays = list(map(float, values(hulls, nodes["AliveObjectData"], "DestructionDelay")))
        print(f"DestructionDelays: {destruction_delays}\n")

        effect_multipliers_kill = list(map(float, values(hulls, nodes["AliveObjectData"], "EffectMultiplierKill")))
        print(f"EffectMultipliersKill: {effect_multipliers_kill}\n")

        effect_multipliers_destroy = list(map(float, values(hulls, nodes["AliveObjectData"], "EffectMultiplierDestroy")))
        print(f"EffectMultipliersDestroy: {effect_multipliers_destroy}\n")

        effects_masks_kill = values(hulls, nodes["AliveObjectData"], "EffectsMaskKill")
        print(f"EffectsMasksKill: {effects_masks_kill}\n")

        effects_masks_destroy = values(hulls, nodes["AliveObjectData"], "EffectsMaskDestroy")
        print(f"EffectsMasksDestroy: {effects_masks_destroy}\n")

        # ComponentData
        hull_index_parents = list(map(int, values(hulls, nodes["ComponentData"], "HullIndexParent")))
        print(f"HullIndexParents: {hull_index_parents}\n")

        local_directions_x = values(hulls, nodes["ComponentData"], "LocalDirectionX")
        local_directions_y = values(hulls, nodes["ComponentData"], "LocalDirectionY")
        local_directions_z = values(hulls, nodes["ComponentData"], "LocalDirectionZ")
        local_directions = [Vector3(local_directions_x[i], local_directions_y[i], local_directions_z[i]) for i in range(len(hulls))]
        print(f"LocalDirections: {local_directions}\n")

        hulls_ = [Hull(hull_types[i],
                       scales[i], positions[i], orientations[i],
                       object_masses[i], object_materials[i],
                       object_maximum_lifes[i], object_armors[i],
                       destruction_delays[i], effect_multipliers_kill[i], effect_multipliers_destroy[i],
                       effects_masks_kill[i], effects_masks_destroy[i],
                       hull_index_parents[i], local_directions[i])
                     for i in range(len(hulls))]

        for i in range(len(hulls)):
            dataset.append({"Name": f"{o}-baseline",
                    "HullIndexParent": hulls_[i].hull_index_parent,
                    "HullType": hulls_[i].hull_type,
                    "ScaleX": hulls_[i].scale.x,
                    "ScaleY": hulls_[i].scale.y,
                    "ScaleZ": hulls_[i].scale.z,
                    "PositionX": hulls_[i].position.x,
                    "PositionY": hulls_[i].position.y,
                    "PositionZ": hulls_[i].position.z,
                    "OrientationW": hulls_[i].orientation.w,
                    "OrientationX": hulls_[i].orientation.x,
                    "OrientationY": hulls_[i].orientation.y,
                    "OrientationZ": hulls_[i].orientation.z,
                    "ObjectMass": hulls_[i].object_mass,
                    "ObjectMaterial": hulls_[i].object_material,
                    "ObjectMaximumLife": hulls_[i].object_maximum_life,
                    "ObjectArmor": hulls_[i].object_armor,
                    "DestructionDelay": hulls_[i].destruction_delay,
                    "EffectMultiplierKill": hulls_[i].effect_multiplier_kill,
                    "EffectMultiplierDestroy": hulls_[i].effect_multiplier_destroy,
                    "EffectsMaskKill": hulls_[i].effects_mask_kill,
                    "EffectsMaskDestroy": hulls_[i].effects_mask_destroy,
                    "LocalDirectionX": hulls_[i].local_direction.x,
                    "LocalDirectionY": hulls_[i].local_direction.y,
                    "LocalDirectionZ": hulls_[i].local_direction.z,
                    "Fitness": 1})

'''
2nd step: Construction of the dataset for the chosen relation with input from all the XML models analyzed.
'''

datasets["hulls"] = dataset

'''
3rd step: Dataset persistance in a CSV format file.
'''

with open(f"{path_out}/datasets/baseline/dataset_hulls.csv", mode='w') as file:
    fieldnames = ["Name",
                  "HullIndexParent",
                  "HullType",
                  "ScaleX",
                  "ScaleY",
                  "ScaleZ",
                  "PositionX",
                  "PositionY",
                  "PositionZ",
                  "OrientationW",
                  "OrientationX",
                  "OrientationY",
                  "OrientationZ",
                  "ObjectMass",
                  "ObjectMaterial",
                  "ObjectMaximumLife",
                  "ObjectArmor",
                  "DestructionDelay",
                  "EffectMultiplierKill",
                  "EffectMultiplierDestroy",
                  "EffectsMaskKill",
                  "EffectsMaskDestroy",
                  "LocalDirectionX",
                  "LocalDirectionY",
                  "LocalDirectionZ",
                  "Fitness"]

    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for row in dataset:
        # print(f"{row}\n")
        writer.writerow(row)



# 1.1.2. Weapon.

'''
1st step: Parsing of XML models.
'''

dataset = []

for o in objects:
    xml = ET.parse(f"{path_in}{o}.xml").getroot()
    
    print(f"###### Working with {o}.xml #######")
    
    ### Weapons ###
    weapons = xml.findall("WEAPONRYWEAPONSAI/Weapon")
    print(f"\nNo. of 'WEAPONRYWEAPONSAI/Weapon': %d\n" % len(weapons))
    
    if len(weapons) > 0:
        weapon_types = list(map(int, [weapon.get("WeaponType") for weapon in weapons]))
        print(f"WeaponTypes: {weapon_types}\n")
        
        # ObjectData
        scales_x = values(weapons, nodes["ObjectData"], "ScaleX")
        scales_y = values(weapons, nodes["ObjectData"], "ScaleY")
        scales_z = values(weapons, nodes["ObjectData"], "ScaleZ")
        scales = [Vector3(scales_x[i], scales_y[i], scales_z[i]) for i in range(len(weapons))]
        print(f"Scales: {scales}\n")

        positions_x = values(weapons, nodes["ObjectData"], "PositionX")
        positions_y = values(weapons, nodes["ObjectData"], "PositionY")
        positions_z = values(weapons, nodes["ObjectData"], "PositionZ")
        positions = [Vector3(positions_x[i], positions_y[i], positions_z[i]) for i in range(len(weapons))]
        print(f"Positions: {positions}\n")

        orientations_w = values(weapons, nodes["ObjectData"], "OrientationW")
        orientations_x = values(weapons, nodes["ObjectData"], "OrientationX")
        orientations_y = values(weapons, nodes["ObjectData"], "OrientationY")
        orientations_z = values(weapons, nodes["ObjectData"], "OrientationZ")
        orientations = [Quaternion(orientations_w[i], orientations_x[i], orientations_y[i], orientations_z[i]) for i in range(len(weapons))]
        print(f"Orientations: {orientations}\n")
        
        # PhysicalObjectData
        object_masses = list(map(float, values(weapons, nodes["PhysicalObjectData"], "ObjectMass")))
        print(f"ObjectMasses: {object_masses}\n") 
        
        object_materials = list(map(int, values(weapons, nodes["PhysicalObjectData"], "ObjectMaterial")))
        print(f"ObjectMaterials: {object_materials}\n") 
        
        # AliveObjectData
        object_maximum_lifes = list(map(float, values(weapons, nodes["AliveObjectData"], "ObjectMaximumLife")))
        print(f"ObjectMaximumLifes: {object_maximum_lifes}\n") 
        
        object_armors = list(map(float, values(weapons, nodes["AliveObjectData"], "ObjectArmor")))
        print(f"ObjectArmors: {object_armors}\n") 
        
        destruction_delays = list(map(float, values(weapons, nodes["AliveObjectData"], "DestructionDelay")))
        print(f"DestructionDelays: {destruction_delays}\n")        
        
        effect_multipliers_kill = list(map(float, values(weapons, nodes["AliveObjectData"], "EffectMultiplierKill")))
        print(f"EffectMultipliersKill: {effect_multipliers_kill}\n")
        
        effect_multipliers_destroy = list(map(float, values(weapons, nodes["AliveObjectData"], "EffectMultiplierDestroy")))
        print(f"EffectMultipliersDestroy: {effect_multipliers_destroy}\n")        
        
        effects_masks_kill = values(weapons, nodes["AliveObjectData"], "EffectsMaskKill")
        print(f"EffectsMasksKill: {effects_masks_kill}\n")        
        
        effects_masks_destroy = values(weapons, nodes["AliveObjectData"], "EffectsMaskDestroy")
        print(f"EffectsMasksDestroy: {effects_masks_destroy}\n")

        # ComponentData
        hull_index_parents = list(map(int, values(weapons, nodes["ComponentData"], "HullIndexParent")))
        print(f"HullIndexParents: {hull_index_parents}\n")

        local_directions_x = values(weapons, nodes["ComponentData"], "LocalDirectionX")
        local_directions_y = values(weapons, nodes["ComponentData"], "LocalDirectionY")
        local_directions_z = values(weapons, nodes["ComponentData"], "LocalDirectionZ")
        local_directions = [Vector3(local_directions_x[i], local_directions_y[i], local_directions_z[i]) for i in range(len(weapons))]
        print(f"LocalDirections: {local_directions}\n")
        
        # WeaponData
        weapons_data = xml.findall("WEAPONRYWEAPONSAI/Weapon/WeaponData")
        
        weapon_priorities = list(map(int,   [weapon_data.get("WeaponPriority") for weapon_data in weapons_data]))
        weapon_behaviours = list(map(int,   [weapon_data.get("WeaponBehaviour") for weapon_data in weapons_data]))
        reload_times      = list(map(float, [weapon_data.get("ReloadTime") for weapon_data in weapons_data]))
        has_locks         = list(map(int,   [weapon_data.get("HasLock") for weapon_data in weapons_data]))
        
        affector_types         = list(map(int,   [weapon_data.get("AffectorType") for weapon_data in weapons_data]))
        affector_scale_factors = list(map(float, [weapon_data.get("AffectorScaleFactor") for weapon_data in weapons_data]))
        affector_power_factors = list(map(float, [weapon_data.get("AffectorPowerFactor") for weapon_data in weapons_data]))
        
        affector_positions_x = list(map(float, [weapon_data.get("AffectorPositionX") for weapon_data in weapons_data]))
        affector_positions_y = list(map(float, [weapon_data.get("AffectorPositionY") for weapon_data in weapons_data]))
        affector_positions_z = list(map(float, [weapon_data.get("AffectorPositionZ") for weapon_data in weapons_data]))
        affector_positions = [Vector3(affector_positions_x[i], affector_positions_y[i], affector_positions_z[i]) for i in range(len(weapons))]

        weapons_ = [Weapon(weapon_types[i],
                           scales[i], positions[i], orientations[i],
                           object_masses[i], object_materials[i],
                           object_maximum_lifes[i], object_armors[i],
                           destruction_delays[i], effect_multipliers_kill[i], effect_multipliers_destroy[i],
                           effects_masks_kill[i], effects_masks_destroy[i],
                           hull_index_parents[i], local_directions[i],
                           weapon_priorities[i], weapon_behaviours[i], reload_times[i], has_locks[i],
                           affector_types[i], affector_scale_factors[i], affector_power_factors[i], affector_positions[i])
                     for i in range(len(weapons))]
        
        for i in range(len(weapons)):
            dataset.append({"Name": f"{o}-baseline",
                    "HullIndexParent": weapons_[i].hull_index_parent,
                    "WeaponType": weapons_[i].weapon_type,
                    "ScaleX": weapons_[i].scale.x,
                    "ScaleY": weapons_[i].scale.y,
                    "ScaleZ": weapons_[i].scale.z,
                    "PositionX": weapons_[i].position.x,
                    "PositionY": weapons_[i].position.y,
                    "PositionZ": weapons_[i].position.z,
                    "OrientationW": weapons_[i].orientation.w,
                    "OrientationX": weapons_[i].orientation.x,
                    "OrientationY": weapons_[i].orientation.y,
                    "OrientationZ": weapons_[i].orientation.z,
                    "ObjectMass": weapons_[i].object_mass,
                    "ObjectMaterial": weapons_[i].object_material,
                    "ObjectMaximumLife": weapons_[i].object_maximum_life,
                    "ObjectArmor": weapons_[i].object_armor,
                    "DestructionDelay": weapons_[i].destruction_delay,
                    "EffectMultiplierKill": weapons_[i].effect_multiplier_kill,
                    "EffectMultiplierDestroy": weapons_[i].effect_multiplier_destroy,
                    "EffectsMaskKill": weapons_[i].effects_mask_kill,
                    "EffectsMaskDestroy": weapons_[i].effects_mask_destroy,
                    "LocalDirectionX": weapons_[i].local_direction.x,
                    "LocalDirectionY": weapons_[i].local_direction.y,
                    "LocalDirectionZ": weapons_[i].local_direction.z,
                    "WeaponPriority": weapons_[i].weapon_priority,
                    "WeaponBehaviour": weapons_[i].weapon_behaviour,
                    "ReloadTime": weapons_[i].reload_time,
                    "HasLock": weapons_[i].has_lock,
                    "AffectorType": weapons_[i].affector_type,
                    "AffectorScaleFactor": weapons_[i].affector_scale_factor,
                    "AffectorPowerFactor": weapons_[i].affector_power_factor,
                    "AffectorPositionX": weapons_[i].affector_position.x,
                    "AffectorPositionY": weapons_[i].affector_position.y,
                    "AffectorPositionZ": weapons_[i].affector_position.z,
                    "Fitness": 1})

'''
2nd step: Construction of the dataset for the chosen relation with input from all the XML models analyzed.
'''

datasets["weapons"] = dataset

'''
3rd step: Dataset persistance in a CSV format file.
'''

with open(f"{path_out}/datasets/baseline/dataset_weapons.csv", mode='w') as file:
    fieldnames = ["Name",
                  "HullIndexParent",
                  "WeaponType",
                  "ScaleX",
                  "ScaleY",
                  "ScaleZ",
                  "PositionX",
                  "PositionY",
                  "PositionZ",
                  "OrientationW",
                  "OrientationX",
                  "OrientationY",
                  "OrientationZ",
                  "ObjectMass",
                  "ObjectMaterial",
                  "ObjectMaximumLife",
                  "ObjectArmor",
                  "DestructionDelay",
                  "EffectMultiplierKill",
                  "EffectMultiplierDestroy",
                  "EffectsMaskKill",
                  "EffectsMaskDestroy",
                  "LocalDirectionX",
                  "LocalDirectionY",
                  "LocalDirectionZ",
                  "WeaponPriority",
                  "WeaponBehaviour",
                  "ReloadTime",
                  "HasLock",
                  "AffectorType",
                  "AffectorScaleFactor",
                  "AffectorPowerFactor",
                  "AffectorPositionX",
                  "AffectorPositionY",
                  "AffectorPositionZ",
                  "Fitness"]

    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for row in dataset:
        # print(f"{row}\n")
        writer.writerow(row)



# 1.1.3. Link.

'''
1st step: Parsing of XML models.
'''

dataset = []

for o in objects:
    xml = ET.parse(f"{path_in}{o}.xml").getroot()
    
    print(f"###### Working with {o}.xml #######")
    
    ### Links ###
    links = xml.findall("HULL/LINKS/Link")
    print(f"\nNo. of 'HULL/LINKS/Link': %d\n" % len(links))
    
    if len(links) > 0:
        link_types = list(map(int, [link.get("LinkType") for link in links]))
        print(f"LinkTypes: {link_types}\n")
        
        hulls_index_first = list(map(int, values(links, nodes["LinkData"], "HullIndexFirst")))
        print(f"HullsIndexFirst: {hulls_index_first}\n")
        
        hulls_index_second = list(map(int, values(links, nodes["LinkData"], "HullIndexSecond")))
        print(f"HullsIndexSecond: {hulls_index_second}\n")        
        
        destructibles = list(map(int, values(links, nodes["LinkData"], "Destructible")))
        print(f"Destructibles: {destructibles}\n")        
        
        destruction_delays = list(map(float, values(links, nodes["LinkData"], "DestructionDelay")))
        print(f"DestructionDelays: {destruction_delays}\n")        
        
        effect_multipliers_kill = list(map(float, values(links, nodes["LinkData"], "EffectMultiplierKill")))
        print(f"EffectMultipliersKill: {effect_multipliers_kill}\n")
        
        effect_multipliers_destroy = list(map(float, values(links, nodes["LinkData"], "EffectMultiplierDestroy")))
        print(f"EffectMultipliersDestroy: {effect_multipliers_destroy}\n")        
        
        effects_masks_kill = values(links, nodes["LinkData"], "EffectsMaskKill")
        print(f"EffectsMasksKill: {effects_masks_kill}\n")        
        
        effects_masks_destroy = values(links, nodes["LinkData"], "EffectsMaskDestroy")
        print(f"EffectsMasksDestroy: {effects_masks_destroy}\n")
    
        links_ = [Link(link_types[i], hulls_index_first[i], hulls_index_second[i],
                       destructibles[i], destruction_delays[i],
                       effect_multipliers_kill[i], effect_multipliers_destroy[i],
                       effects_masks_kill[i], effects_masks_destroy[i])
                  for i in range(len(links))]
        
        for i in range(len(links)):
            dataset.append({"Name": f"{o}-baseline",
                    "LinkType": links_[i].link_type,
                    "HullIndexFirst": links_[i].hull_index_first,
                    "HullIndexSecond": links_[i].hull_index_second,
                    "Destructible": links_[i].destructible,
                    "DestructionDelay": links_[i].destruction_delay,
                    "EffectMultiplierKill": links_[i].effect_multiplier_kill,
                    "EffectMultiplierDestroy": links_[i].effect_multiplier_destroy,
                    "EffectsMaskKill": links_[i].effects_mask_kill,
                    "EffectsMaskDestroy": links_[i].effects_mask_destroy,
                    "Fitness": 1})

'''
2nd step: Construction of the dataset for the chosen relation with input from all the XML models analyzed.
'''

datasets["links"] = dataset

'''
3rd step: Dataset persistance in a CSV format file.
'''

with open(f"{path_out}/datasets/baseline/dataset_links.csv", mode='w') as file:
    fieldnames = ["Name",
                  "LinkType",
                  "HullIndexFirst",
                  "HullIndexSecond",
                  "Destructible",
                  "DestructionDelay",
                  "EffectMultiplierKill",
                  "EffectMultiplierDestroy",
                  "EffectsMaskKill",
                  "EffectsMaskDestroy",
                  "Fitness"]

    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for row in dataset:
        # print(f"{row}\n")
        writer.writerow(row)

###### Working with Argos.xml #######

No. of 'HULL/Hull': 36

HullTypes: [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Scales: [Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.0, 1.0), Vector3(1.0, 1.

## 1.2. Association relationships (4)

### 1.2.1. Link-HullIndexFirst-Hull
### 1.2.2. Link-HullIndexSecond-Hull
### 1.2.3. Weapon-MobileObject
### 1.2.4. Weapon-Cannon

# 2. Mutant creation for training

In [ ]:
for key in datasets.keys():
    ### Aggregation relationships
    # TODO: if "hulls" in key:
    if "links" in key:
        fieldnames = ["Name", "LinkType", "HullIndexFirst", "HullIndexSecond", "Destructible", "DestructionDelay",
                      "EffectMultiplierKill", "EffectMultiplierDestroy", "EffectsMaskKill", "EffectsMaskDestroy",
                      "Fitness"]
        mutate_model(datasets[key], props=["DestructionDelay", "EffectMultiplierKill", "EffectMultiplierDestroy"], percentage=[10, 15], times=49, filename=f"{key}_mut", fieldnames=fieldnames)
    elif "weapons" in key:
        fieldnames = ["Name", "HullIndexParent",
                      "ScaleX", "ScaleY", "ScaleZ", "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ",
                      "Fitness"]
        mutate_model(datasets[key], props=["ScaleX", "ScaleY", "ScaleZ", "PositionX", "PositionY", "PositionZ",
                                           "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                                           "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ"], percentage=[10, 15], times=49, filename=f"{key}_mut", fieldnames=fieldnames)
    elif "ai_units" in key:
        fieldnames = ["Name", "UnitValue"] + list(behaviour_unit_ai_data_[0].keys()) + ["Fitness"]
        mutate_model(datasets[key], props=["AIBehaviourDistance", "AIAttackSpeed"], percentage=[10, 15], times=49, filename=f"{key}_mut", fieldnames=fieldnames)
    elif "ai_movements" in key:
        fieldnames = ["Name"] + list(movement_ai_.keys()) + ["Fitness"]
        mutate_model(datasets[key], props=["Acceleration", "MinimumSpeed", "MaximumSpeed", "ReachDestinationAtFullLinearSpeed", "AccelerationAngular", "MaximumSpeedAngular", "ReachDestinationAtFullAngularSpeed"], percentage=[10, 15], times=49, filename=f"{key}_mut", fieldnames=fieldnames)
    ### Assocation relationships
    # TODO: elif "links_hif_hull" in key: && elif "links_his_hull" in key:
    elif "weapons_mobile_objects" in key:
        fieldnames = ["Name", "HullIndexParent",
                      "ScaleX", "ScaleY", "ScaleZ",
                      "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ",
                      "MobilityType", "LocalPositionX", "LocalPositionY", "LocalPositionZ",
                      "Fitness"]
        mutate_model(datasets[key], props=["LocalDirectionX", "LocalDirectionY", "LocalDirectionZ", "LocalPositionX", "LocalPositionY", "LocalPositionZ"], percentage=[10, 15], times=49, filename=f"{key}_mut", fieldnames=fieldnames)
    elif "weapons_cannons" in key:
        fieldnames = ["Name", "HullIndexParent",
                      "ScaleX", "ScaleY", "ScaleZ",
                      "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ",
                      "FiringTurnMaximum", "FiringTurnStart", "FiringTurnEnd", "Roll",
                      "FiringPositionX","FiringPositionY","FiringPositionZ",
                      "FiringDirectionX","FiringDirectionY","FiringDirectionZ",
                      "Fitness"]
        mutate_model(datasets[key], props=["LocalDirectionX", "LocalDirectionY", "LocalDirectionZ", "FiringTurnMaximum", "FiringTurnStart", "FiringTurnEnd", "Roll", "FiringPositionX", "FiringPositionY", "FiringPositionZ", "FiringDirectionX", "FiringDirectionY", "FiringDirectionZ"], percentage=[10, 15], times=49, filename=f"{key}_mut", fieldnames=fieldnames)

# 3. Model classification (10-fold Cross-validation)

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import seaborn as sb
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
for key in datasets.keys():
    if "weapons" in key:
        fieldnames = ["ScaleX", "ScaleY", "ScaleZ", "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ"]
    elif "links" in key:
        # TODO: Define props to be mutated
        fieldnames = ["DestructionDelay", "EffectMultiplierKill", "EffectMultiplierDestroy"]
    
    print(f"### {key}_mut.csv ###\n")
    model_data = pd.read_csv(f"{key}_mut.csv")

    all_inputs = model_data[fieldnames].values
    all_classes = model_data["Name"].values
    
    print(all_inputs[:5], end='\n\n')
    print(f"Classes: {all_classes}", end='\n\n')
    
    (training_inputs,
     testing_inputs,
     training_classes,
     testing_classes) = train_test_split(all_inputs, all_classes, train_size=0.80, random_state=456)

    print(f"Training set: {training_inputs.shape}")
    print(f"Testing set: {testing_inputs.shape}", end='\n\n')
    
    ### DecisionTreeClassifier ###
    # Create the classifier.
    decision_tree_classifier = DecisionTreeClassifier()
    # Train the classifier on the training set.
    decision_tree_classifier.fit(training_inputs, training_classes)
    # Validate the classifier on the testing set using classification accuracy.
    print("DecisionTreeClassifier")
    print(decision_tree_classifier.score(testing_inputs, testing_classes))
    print(decision_tree_classifier.predict(testing_inputs[:1, :]), end='\n\n')

    ### Model accuracies ###
    plt.title("Model accuracies")
    # DecisionTreeClassifier
    model_accuracies = []
    for repetition in range(1000):
        (training_inputs,
         testing_inputs,
         training_classes,
         testing_classes) = train_test_split(all_inputs, all_classes, train_size=0.75)

        decision_tree_classifier = DecisionTreeClassifier()
        decision_tree_classifier.fit(training_inputs, training_classes)
        
        classifier_accuracy = decision_tree_classifier.score(testing_inputs, testing_classes)
        model_accuracies.append(classifier_accuracy)
    sb.distplot(model_accuracies, label="DecisionTreeClassifier")
    plt.legend()
    plt.savefig(f"plots/{key}_model_accuracies.pdf")
    plt.close("all")
    
    ### 10-fold Cross-validation ###
    # DecisionTreeClassifier(max_depth=4)
    decision_tree_classifier = DecisionTreeClassifier()

    # cross_val_score returns a list of the scores, which we can visualize
    # to get a reasonable estimate of the classifier's performance
    cv_scores = cross_val_score(decision_tree_classifier, all_inputs, all_classes, cv=10)
    sb.distplot(cv_scores)
    plt.title(f"Average score: {np.mean(cv_scores)}")
    plt.savefig(f"plots/{key}_model_cv_scores.pdf")
    plt.close("all")